<h2>Importación de librerias</h2>

In [33]:
# Visualization Libraries
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

# Operational Libraries
import pandas as pd
import re
import dask.dataframe as dd

<h2>1. Carga de datos desde excel a dataframes de pandas</h2>


En primer lugar voy a cargar cada uno de los csv a unos dataframes de pandas para ir manejandolos desde aqui. La idea es ir cruzando algunos excels y ver qué graficos interesantes van saliendo de los datos que tenemos. Los voy a cargar todos de una vez y a continuación los veremos en tablas para ir describiendolos

Esto puede tomar un ratito

In [3]:
df_informacion_de_clientes = pd.read_csv('Informacion_Clientes.csv')
df_simulaciones_clientes = pd.read_csv('Simulaciones_Clientes.csv')
df_tratamiento = pd.read_csv('Tratamiento.csv')
df_ventas = pd.read_csv('Ventas.csv')

<h3>1.1 Carga de 'Informacion_Clientes.csv'</h3>

En primer lugar se cargará la información de los clientes. Esto incluye las siguientes características de los clientes:

* **unnamed**: algo como uid
* **Rut**: identificador de Chile (supongo que por privacidad va desde 0 a max de observaciones)
* **Género**: Masculino o femenino
* **Categoría_Digital**: Si el cliente es digital o no
* **Elasticidad_Precios**: Baja, media o alta
* **Nacionalidad**: Chileno o extranjero
* **Propensión**: Número entre 0 y 1 que idica ??
* **Probabilidad_No_Pago**: Número entre 0 y 1 que indica la probabilidad de que el cliente no pague la deuda
* **Edad**: Numero entero de edad en años
* **Renta**: Renta promedio de los últimos 12 meses
* **Oferta_Consumo**: Monto máximo que puede cursar un cliente dado sus antecedentes crediticios y situación socioeconómica. 
* **Deuda_CMF**: Deuda que tiene el cliente en otros bancos. Efectivamente es deuda pendiente, pero de créditos otorgados por la competencia.
* **Tiempo_como_cliente**: Número de tiempo(no sé en que medida está) que el cliente lleva en el banco


In [4]:
df_informacion_de_clientes

,Unnamed: 0,rut,Genero,Categoria_Digital,Elasticidad_Precios,Nacionalidad,Propension,Probabilidad_No_Pago,Edad,Renta,Oferta_Consumo,Deuda_CMF,Tiempo_como_cliente
0,0,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,6.258183e+05,2164276.0,7.125854e+05,2
1,1,2,Femenino,Cliente no Digital,Baja,Chileno,0.291601,0.014320,41.0,3.172616e+05,1451704.0,1.970697e+05,2
2,2,3,Femenino,Cliente no Digital,Baja,Chileno,0.685085,0.002156,38.0,1.240551e+07,39322607.0,1.400604e+07,2
3,3,4,Masculino,Cliente no Digital,Alta,Chileno,0.914672,0.034418,57.0,5.441466e+05,1607042.0,5.159465e+05,2
4,4,5,Masculino,Cliente Digital,Media,Chileno,0.425077,0.014978,26.0,1.870225e+05,493865.0,2.601013e+05,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
543646,543646,543647,Femenino,Cliente Digital,Baja,Chileno,0.144657,0.037291,29.0,1.176598e+05,9625991.0,1.299826e+05,2
543647,543647,543648,Masculino,Cliente no Digital,Baja,Chileno,0.740170,0.035877,31.0,1.558612e+06,7522126.0,1.740917e+06,2
543648,543648,543649,Masculino,Cliente no Digital,Media,Chileno,0.255285,0.023306,49.0,9.449508e+05,1889172.0,1.057952e+06,2
543649,543649,543650,Femenino,Cliente no Digital,Media,Chileno,0.709086,0.015121,40.0,1.039964e+06,4518498.0,1.010872e+06,2


In [5]:
df_informacion_de_clientes['Genero'].nunique()

2

In [6]:
tiempo_como_cliente_maximo = df_informacion_de_clientes['Tiempo_como_cliente'].max()
tiempo_como_cliente_minimo =df_informacion_de_clientes['Tiempo_como_cliente'].max()
print(f'Tiempo máximo como cliente: {tiempo_como_cliente_maximo}\nTiempo mínimo como cliente: {tiempo_como_cliente_minimo}')

Tiempo máximo como cliente: 2
Tiempo mínimo como cliente: 2


Algo interesante es que tiempo como cliente es solamente '2'

<h3>1.2 Carga de 'Simulaciones_Clientes.csv'</h3>

En segundo lugar se cargaran las simulaciones hechas por los clientes en la página del banco. Esto incluye las siguientes características de las simulaciones:
* **unnamed**: Supongo que es el número de simulacion registrada, un tipo de identificador de la simulación
* **fecha**: yyyy-mm-dd fecha de la simulación
* **rut**: identificador de Chile del cliente que hizo la simulacion
* **monto_simulado**: monto prestado al cliente
* **plazo_simulado**: plazo en **meses** del crédito
* **tasa_simulado**: costo para el cliente del credito

In [7]:
df_simulaciones_clientes

,Unnamed: 0,fecha,rut,Monto_Simulado,Plazo_Simulado,Tasa_Simulado
0,0,2019-01-01,3,29881539,29,1.441744
1,1,2019-06-01,3,30788278,32,2.016437
2,2,2020-02-01,3,32612203,32,2.297867
3,3,2020-03-01,3,32393538,31,1.589299
4,4,2020-05-01,3,34527366,21,1.184410
...,...,...,...,...,...,...
8528022,8528022,2024-02-01,387808,7761228,31,1.971523
8528023,8528023,2024-04-01,387808,8534288,36,1.349430
8528024,8528024,2024-03-01,115711,3080827,20,2.215447
8528025,8528025,2024-04-01,62510,1403394,38,1.552448


In [8]:
df_simulaciones_clientes[df_simulaciones_clientes['rut'] == 9]

,Unnamed: 0,fecha,rut,Monto_Simulado,Plazo_Simulado,Tasa_Simulado
3871691,3871691,2019-03-01,9,19099,29,1.529443
3871692,3871692,2019-08-01,9,18215,32,1.862135
3871693,3871693,2020-01-01,9,16175,31,2.069854
3871694,3871694,2020-03-01,9,15730,38,1.909511
3871695,3871695,2020-06-01,9,20712,31,1.291823
3871696,3871696,2020-08-01,9,18897,32,2.175747
3871697,3871697,2021-02-01,9,19728,35,2.478797
3871698,3871698,2021-03-01,9,17941,35,1.604788
3871699,3871699,2021-08-01,9,16918,40,1.552956
3871700,3871700,2022-02-01,9,19111,34,1.753733


Lo interesante es que con el rut se pueden cruzar el dataframe de informacion de cliente y del plazo simulado

Para futuro manejo voy a renombrar la columna 'fecha'

In [28]:
df_simulaciones_clientes.rename(columns={'fecha': 'fecha_simulacion'}, inplace=True)

<h3>1.3 Carga de 'Tratamiento.csv'</h3>

En tercer lugar se cargara el tratamiento que ha tenido el banco con el cliente, es decir, cómo se han contactado con él. Esto incluye las siguientes características:

* **unnamed**: Número de tratamiento registrado
* **fecha**: yyyy-mm-dd
* **rut**: Identificador de Chile del cliente con el que se tiene el tipo de trato
* **n_correos**: Cantidad de correos que se enviaron en el mes que sale la fecha. Es decir, si sele fecha '2024-03-01', correspondería a los correos enviados en marzo de 2024.
* **asg_ejec**: Si el cliente tiene un ejecutivo asignado

-[Notas de jp]: Este es el que más me costo entender, está registrado el tratamiento por fecha. Hay muchos más tratamientos(3 millones y medio) que clientes(500 mil clientes) por lo que supongo que el tipo de trato que le dan al cliente pudo haber cambiado en el tiempo. No estoy seguro si la cantidad de correos es un recuento del total de correos hasta la fecha o cuantos se les envió esa fecha.

In [9]:
df_tratamiento

,Unnamed: 0,fecha,rut,n_correos,asg_ejec
0,0,2019-01-01,1,3,0
1,1,2019-01-01,2,4,0
2,2,2019-01-01,3,3,0
3,3,2019-01-01,4,1,1
4,4,2019-01-01,5,2,1
...,...,...,...,...,...
35880961,543646,2024-06-01,543647,1,0
35880962,543647,2024-06-01,543648,4,0
35880963,543648,2024-06-01,543649,0,0
35880964,543649,2024-06-01,543650,4,0


Para futuro manejo voy a renombrar la columna 'fecha'

In [26]:
df_tratamiento.rename(columns={'fecha': 'fecha_tratamiento'}, inplace=True)

<h3>1.4 Carga de 'Ventas.csv'</h3>

Por último se cargaran las ventas que ha tenido el banco con el cliente. Esto incluye las siguientes características:

* **unnamed**: Índice sin significado
* **fecha**: yyyy-mm-dd -> fecha en la que se concretó la venta
* **rut**: identificador de Chile del cliente al que se le concretó la venta

In [10]:
df_ventas

,Unnamed: 0,fecha,rut
0,4,2020-05-01,3
1,6,2020-12-01,3
2,9,2021-12-01,3
3,17,2023-07-01,3
4,22,2019-04-01,11
...,...,...,...
2807559,8528011,2023-11-01,46089
2807560,8528012,2024-01-01,46089
2807561,8528020,2024-02-01,64855
2807562,8528021,2024-04-01,64855


Para futuro manejo voy a renombrar la columna 'fecha'

In [27]:
df_ventas.rename(columns={'fecha': 'fecha_venta'}, inplace=True)

<h2>2. Joints de datos<h2>


Ya con todos los df cargados se pueden ir cruzando algunos de ellos para poder ir sacando algunas conclusiones.

El primer joint que haré es la informacion de los clientes y la simulacion. 

Quiero saber: de todos mis clientes quienes son los que simulan. Por lo tanto, para cada uno de los clientes que exista en la base de informacion de clientes quiero tener la informacion de sus simulaciones (o no simulaciones es que no han simulado). Esto es interesante para saber si simulan o no.

In [29]:
df_simulaciones_e_informacion_de_clientes = pd.merge(df_informacion_de_clientes, df_simulaciones_clientes, on='rut', how='left')

Revisaremos si hay algun NaN en este merge, es decir, si algún cliente no ha simulado.

In [30]:
df_simulaciones_e_informacion_de_clientes['Monto_Simulado'].isna().sum()

4

Con esto, podemos ver que hay 4 clientes de nuestros datos que no han simulado. 

No lo eliminare para ver si es que a pesar de no haber simulado aún asi el banco ha concretado una venta con ellos.

A continuación haré un procedimiento similar con las ventas

In [34]:
df_simulaciones_e_informacion_de_clientes_ventas = pd.merge(df_simulaciones_e_informacion_de_clientes, df_ventas, on = 'rut', how='left')

MemoryError: Unable to allocate 37.1 MiB for an array with shape (4862678, 1) and data type float64

In [ ]:
df_simulaciones_e_informacion_de_clientes_ventas

,Unnamed: 0_x,rut,Genero,Categoria_Digital,Elasticidad_Precios,Nacionalidad,Propension,Probabilidad_No_Pago,Edad,Renta,Oferta_Consumo,Deuda_CMF,Tiempo_como_cliente,Unnamed: 0_y,fecha_x,Monto_Simulado,Plazo_Simulado,Tasa_Simulado,Unnamed: 0,fecha_y
0,0,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2,7864771.0,2019-10-01,319936.0,33.0,1.092295,7864773.0,2020-02-01
1,0,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2,7864771.0,2019-10-01,319936.0,33.0,1.092295,7864775.0,2021-06-01
2,0,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2,7864771.0,2019-10-01,319936.0,33.0,1.092295,7864779.0,2022-08-01
3,0,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2,7864771.0,2019-10-01,319936.0,33.0,1.092295,7864781.0,2023-04-01
4,0,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2,7864771.0,2019-10-01,319936.0,33.0,1.092295,7864783.0,2023-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48615154,543650,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,2,3871678.0,2024-06-01,349828.0,16.0,1.709522,3871667.0,2021-05-01
48615155,543650,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,2,3871678.0,2024-06-01,349828.0,16.0,1.709522,3871671.0,2022-06-01
48615156,543650,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,2,3871678.0,2024-06-01,349828.0,16.0,1.709522,3871673.0,2023-02-01
48615157,543650,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,2,3871678.0,2024-06-01,349828.0,16.0,1.709522,3871677.0,2024-05-01


In [ ]:
df_simulaciones_e_informacion_de_clientes_ventas['fecha_venta'].isna().sum()

118503

Esto indica que 118.503 simulaciones no resultaron en ventas concretas. Es importante recordar que, debido a las características de este dataset, el hecho de que un cliente aparezca en el archivo CSV de ventas implica que realizó un cursamiento en el crédito. Además, se mencionó que la fecha de cursamiento del crédito debe coincidir con la fecha en la que se realizó la simulación, lo que significa que el monto y el precio registrados corresponden a dicha simulación.

Ahora, lo que me interesa es poder ver de todos los clientes que simularon como distribuyen sus características. Para esto, voy a crear otro dataframe en el cual voy a eliminar algunas columnas y voy a remapear algunas otras de nombres a números.

Se copia el dataset para evitar futuros problemas

In [16]:
dataset_simulacion_informacion_ventas_numerico = df_simulaciones_e_informacion_de_clientes_ventas.copy(deep=True)

<h3>2.1. Valores que no aportan información</h3>

En este caso, los atributos que no aportan informacion son: 'unnamed_x', 'tiempo_como_cliente', 'unnamed:0'

In [22]:
dataset_simulacion_informacion_ventas_numerico = dataset_simulacion_informacion_ventas_numerico.drop(['Unnamed: 0_x', 'Tiempo_como_cliente', 'Unnamed: 0'], axis=1)

In [25]:
dataset_simulacion_informacion_ventas_numerico

,rut,Genero,Categoria_Digital,Elasticidad_Precios,Nacionalidad,Propension,Probabilidad_No_Pago,Edad,Renta,Oferta_Consumo,Deuda_CMF,Unnamed: 0_y,fecha_x,Monto_Simulado,Plazo_Simulado,Tasa_Simulado,fecha_y
0,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,7864771.0,2019-10-01,319936.0,33.0,1.092295,2020-02-01
1,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,7864771.0,2019-10-01,319936.0,33.0,1.092295,2021-06-01
2,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,7864771.0,2019-10-01,319936.0,33.0,1.092295,2022-08-01
3,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,7864771.0,2019-10-01,319936.0,33.0,1.092295,2023-04-01
4,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,7864771.0,2019-10-01,319936.0,33.0,1.092295,2023-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48615154,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,3871678.0,2024-06-01,349828.0,16.0,1.709522,2021-05-01
48615155,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,3871678.0,2024-06-01,349828.0,16.0,1.709522,2022-06-01
48615156,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,3871678.0,2024-06-01,349828.0,16.0,1.709522,2023-02-01
48615157,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,3871678.0,2024-06-01,349828.0,16.0,1.709522,2024-05-01


Recordemos que los datos pueden ser de cuatro tipos principalmente:

- Nominal: nombres (identificación y clasificación)
- Ordinal: orden (jerarquización, posición relativa)
- Intervalo: cuantificación (cero arbitrario)
- Escala: cuantificación (cero absoluto)

<h3>2.2. Valores Nominales</h3>

En este caso los valores nominales